# With De=3e-10,3e-12, and others with Andrew's value

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import traceback
import imageio.v2 as imageio
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : fs}
mpl.rc('font', **font)

In [2]:
V_max = 4.2;        V_min = 2.5;
total_cycle = 500; save_at_cycles= 1
Experiment_short   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 1 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [3]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R7/'; 
Target  = 'a1_4_dd_De/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [4]:
model = pybamm.lithium_ion.DFN(     
    options={  
        "solvent diffusion": "EC wo refill"     } )
De_all = [3e-10,3e-11,3e-12]
Dcross  = 3e-11
Dec     = 5e-10

using EC wo refill for Li+


Para_0.search('mole')

In [5]:

ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008Exp";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0['EC transference number'] =     -1.4
Para_0['Cation transference number'] =     0.28

Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = Dcross
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =  Dcross

#print(Model_d.options["solvent diffusion"])
Para_0['Ratio of lithium moles to SEI moles'] =  1

c_e = model.variables["Electrolyte concentration [mol.m-3]"]
T = model.variables["Cell temperature [K]"]
c_EC = model.variables["EC concentration [mol.m-3]"]
model.variables["c(EC) over c(Li+)"] = c_EC / c_e
model.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e,c_EC, T))
    #model.variables["Electrolyte diffusivity [m2.s-1]"] =(
    #    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e,c_EC, T))

Xi =  list(np.linspace(-1.8, -1.0, 10))
print(type(Xi))

In [6]:
Sim = [];Sol=[]; 
for De in De_all:
    Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  Dec
    Para_0['Electrolyte diffusivity [m2.s-1]'] = De
    sim = pybamm.Simulation(
        model, experiment = Experiment_short,
        parameter_values = Para_0,
        solver = pybamm.CasadiSolver(),)  
    try:
        sol = sim.solve(save_at_cycles=save_at_cycles,);
    except:
        print('Fail for De= ',De)
    else:
        Sol.append(sol)   
        print('Succeed for De= ',De)

Succeed for De=  3e-10


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
At t = 0.00416598 and h = 4.13639e-010, the corrector convergence failed repeatedly or with |h| = hmin.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
At t = 0.00416598 and h = 4.13639e-010, the corrector convergence failed repeatedly or with |h| = hmin.
CasADi -  WARNING("roots:g failed: Inf detected for output x, at (row 20, col 0).") [.../casadi/core/oracle_function.cpp:265]
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
2022-10-21 11:16:17.928 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Maximum number of

Succeed for De=  3e-11


At t = 0.00157305 and h = 8.7545e-012, the corrector convergence failed repeatedly or with |h| = hmin.
2022-10-21 11:16:23.216 - [ERROR] callbacks.on_experiment_error(226): Simulation error: .../casadi/interfaces/sundials/idas_interface.cpp:591: IDASolve returned "IDA_CONV_FAIL". Consult IDAS documentation.


Fail for De=  3e-12


print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

In [7]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 15}
mpl.rc('font', **font)

In [8]:
label = [De for De in De_all]
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Negative electrode potential [V]",
    "Positive electrode potential [V]",
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (12,8)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

ValueError: labels '[3e-10, 3e-11, 3e-12]' have different length to models '['Doyle-Fuller-Newman model', 'Doyle-Fuller-Newman model']'

In [ ]:
label = [De for De in De_all]
output_variables3 = [
    "Terminal voltage [V]",   
    "Discharge capacity [A.h]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (12,8)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.plot(0.5)
#quick_plot.fig.savefig(BasicPath + Target+"t_0+_three_cases.png", dpi=600)
#quick_plot.create_gif(
    #number_of_images=10, duration=2,output_filename=BasicPath + Target+"concentration and flux.gif")

In [ ]:
label = [De for De in De_all]
output_variables3 = [
    #"Terminal voltage [V]",   
    "EC flux [mol.m-2.s-1]",
    "EC flux by diffusion [mol.m-2.s-1]",
    "EC flux by migration [mol.m-2.s-1]",
    "EC flux by Li+ [mol.m-2.s-1]",
    "Li+ flux [mol.m-2.s-1]",
    "Li+ flux by diffusion [mol.m-2.s-1]",
    "Li+ flux by migration [mol.m-2.s-1]",
    "Li+ flux by solvent [mol.m-2.s-1]",      
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (12,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.plot(0.5)
#quick_plot.fig.savefig(BasicPath + Target+"t_0+_three_cases.png", dpi=600)
##quick_plot.create_gif(
#    #number_of_images=10, duration=2,output_filename=BasicPath + Target+"flux contribution.gif")

In [ ]:
label = [De for De in De_all]
output_variables3 = [
    "Electrolyte concentration",
    "Minus div Li+ flux",
    "Li+ source term",
    "Minus div Li+ flux by diffusion",
    "Minus div Li+ flux by migration",
    "Minus div Li+ flux by solvent",
    #"Li+ source term refill",
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (12,9)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
    #number_of_images=10, duration=2,output_filename=BasicPath + Target+"Li+ rhs contribution.gif")

In [ ]:
label = [De for De in De_all]
output_variables3 = [
    "EC concentration",
    "EC source term (SEI)",
    "Minus div EC flux",
    "Minus div EC flux by diffusion",
    "Minus div EC flux by migration",
    "Minus div EC flux by Li+",
    #"Li+ source term refill",
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (12,9)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
    #number_of_images=10, duration=2,output_filename=BasicPath + Target+"EC rhs contribution.gif")